In [3]:
import pandas as pd
import numpy as np
import json
import random
from concurrent.futures import ThreadPoolExecutor
from functools import partial

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch

from rdkit import rdBase
rdBase.DisableLog('rdApp.error') 

from dotenv import load_dotenv
load_dotenv()

from model_2D.model_2D import GNNFingerprint2D
from model_2D.utils import smiles_to_2D_fp

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(parent_dir)

from model.model import GNNFingerprint3D
from hypotheses.compare_with_other_fingerprints.utils import smiles_to_3D

In [4]:
data_path = os.getenv("DATA_PATH")
models_path = os.getenv("MODELS_PATH")

columns = ["mu", "zpve", "energy_U0"]

### 2D Task

In [5]:
def read_data(path, ki_threshold):
    df = pd.read_csv(path, sep=";")
    df['Activity'] = df['Standard Value'].apply(lambda x: 1 if x < ki_threshold else 0)

    df = df[['Smiles', 'Activity']].dropna()

    X_train, X_test, y_train, y_test = train_test_split(
        df['Smiles'], df['Activity'], test_size=0.2, random_state=42
    )

    return X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)

In [6]:
def get_score(X_train, y_train, X_test, y_test, task = "c"):
    s_scaler = StandardScaler()
    X_train = s_scaler.fit_transform(X_train)
    X_test = s_scaler.transform(X_test)

    # PCA to 167 dim
    pca = PCA(n_components=167)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    if task == "c":
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        print("Classification Report:")
        print(classification_report(y_test, y_pred))

    else:
        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        print("Regression Metrics:")
        print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
        print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
        print(f"R²:   {r2_score(y_test, y_pred):.4f}")

In [7]:
fingerprint_model_2D = GNNFingerprint3D(13, 5)
fingerprint_model_2D.load_state_dict(torch.load(os.path.join(models_path, "GNN_MORE_WEIGHT_3D.pth")))
fingerprint_model_2D = fingerprint_model_2D.to("cuda")
fingerprint_model_2D.eval()

fingerprint_model_3D = GNNFingerprint3D(13, 5)
fingerprint_model_3D.load_state_dict(torch.load(os.path.join(models_path, "FINAL_GNN.pth")))
fingerprint_model_3D = fingerprint_model_3D.to("cuda")
fingerprint_model_3D.eval()

model_2D = GNNFingerprint2D(14)
model_2D.load_state_dict(torch.load(os.path.join(models_path, "2D_GNN_MODEL.pth")))
model_2D = model_2D.to("cuda")
model_2D.eval()

with open(os.path.join(data_path, "means_and_stds.json")) as f:
    scaler = json.load(f)

C:\Users\jakub\AppData\Local\Temp\ipykernel_23888\977861933.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fingerprint_model_2D.load_state_dict(torch.load(os.path.join(

In [8]:
folder = os.path.join(data_path, "CHEMBL")
device = "cuda" if torch.cuda.is_available() else "cpu"
scaler_path = "2D_test_min_max.json"

for db in os.listdir(folder):
    print("============================================")
    print(db)
    print("============================================")

    data = os.path.join(folder, db)
    X_train, X_test, y_train, y_test = read_data(data, 100)

    X_train, y_train = list(X_train), list(y_train)
    X_test, y_test = list(X_test), list(y_test)

    ### 2D MODEL
    print("****************")
    print("2D MODEL")
    print("****************")
    X_train_2D = np.array([smiles_to_2D_fp(smiles, model_2D, device, scaler_path).detach().cpu() for smiles in X_train])
    X_test_2D = np.array([smiles_to_2D_fp(smiles, model_2D, device, scaler_path).detach().cpu() for smiles in X_test])

    rows_all_nan_train = np.isnan(X_train_2D).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_2D).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_2D = np.delete(X_train_2D, nan_indices_train, axis=0)
    y_train_2D = np.delete(y_train, nan_indices_train, axis=0)
    X_test_2D = np.delete(X_test_2D, nan_indices_test, axis=0)
    y_test_2D = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_2D, y_train_2D, X_test_2D, y_test_2D)

    ### 2D+3D
    print("****************")
    print("2D+3D MODEL")
    print("*****************")

    X_train_3D = np.array([smiles_to_3D(smiles, fingerprint_model_2D, scaler, False).detach().cpu() for smiles in X_train])
    X_test_3D = np.array([smiles_to_3D(smiles, fingerprint_model_2D, scaler, False).detach().cpu() for smiles in X_test])

    rows_all_nan_train = np.isnan(X_train_3D).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_3D).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_3D = np.delete(X_train_3D, nan_indices_train, axis=0)
    y_train_3D = np.delete(y_train, nan_indices_train, axis=0)
    X_test_3D = np.delete(X_test_3D, nan_indices_test, axis=0)
    y_test_3D = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_3D, y_train_3D, X_test_3D, y_test_3D)


CHEMBL1833_5HT2B.csv
****************
2D MODEL
****************
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.96      0.91       408
           1       0.82      0.50      0.62       134

    accuracy                           0.85       542
   macro avg       0.84      0.73      0.76       542
weighted avg       0.85      0.85      0.84       542

****************
2D+3D MODEL
*****************
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.98      0.90       408
           1       0.87      0.44      0.58       134

    accuracy                           0.85       542
   macro avg       0.85      0.71      0.74       542
weighted avg       0.85      0.85      0.83       542

CHEMBL214_5HT1A.csv
****************
2D MODEL
****************
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.71      0.72     

[11:23:43] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[11:23:43] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[11:23:43] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[11:27:15] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[11:27:15] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[11:27:15] UFFTYPER: Warning: hybridization set to SP3 for atom 29


Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       549
           1       0.76      0.80      0.78       673

    accuracy                           0.75      1222
   macro avg       0.75      0.75      0.75      1222
weighted avg       0.75      0.75      0.75      1222

CHEMBL224_5HT2A.csv
****************
2D MODEL
****************
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       704
           1       0.89      0.71      0.79       532

    accuracy                           0.84      1236
   macro avg       0.85      0.82      0.83      1236
weighted avg       0.84      0.84      0.83      1236

****************
2D+3D MODEL
*****************
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.94      0.87       704
           1       0.89      0.70      0.79       532

   

### 3D TASK

In [9]:
def read_data_3D(db_name, size):
    path = os.path.join(data_path, db_name)
    all_files = [ff for f in os.listdir(path) if os.path.isfile(ff := os.path.join(path, f))]

    random_files = random.sample(all_files, size)
    l = []
    for i, file in enumerate(random_files):
        with open(file, 'r') as f:
            data = json.load(f)
        rec = {"smiles": data["smiles"], "conf": data["atoms"], "homo": data["homo"], "lumo": data["lumo"]}
        for col in columns:
            rec[col] = data[col]
        l.append(rec)

    return pd.DataFrame(l)

In [10]:
df = read_data_3D("qm9_data_json", 20000)
device = "cuda" if torch.cuda.is_available() else "cpu"

for col in columns:
    print("============================================")
    print(col)
    print("============================================")

    X = df[["smiles", "conf", "homo", "lumo"]]
    y = df[col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    
    ### 2D MODEL
    print("****************")
    print("2D MODEL")
    print("****************")
    
    X_train_2D = np.array([smiles_to_2D_fp(smiles[0], model_2D, device, scaler_path).detach().cpu() for smiles in X_train])
    X_test_2D = np.array([smiles_to_2D_fp(smiles[0], model_2D, device, scaler_path).detach().cpu() for smiles in X_test])

    rows_all_nan_train = np.isnan(X_train_2D).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_2D).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_2D = np.delete(X_train_2D, nan_indices_train, axis=0)
    y_train_2D = np.delete(y_train, nan_indices_train, axis=0)
    X_test_2D = np.delete(X_test_2D, nan_indices_test, axis=0)
    y_test_2D = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_2D, y_train_2D, X_test_2D, y_test_2D, "r")

    ### 2D+3D
    print("****************")
    print("2D+3D MODEL")
    print("*****************")

    def get_3D_embedding(smiles, model, scaler):
        return smiles_to_3D(smiles, model, scaler).detach().cpu()

    embedding_fn_3D = partial(get_3D_embedding, model=fingerprint_model_3D, scaler=scaler)

    with ThreadPoolExecutor(max_workers=16) as executor:
        X_train_3D = np.array(list(executor.map(embedding_fn_3D, X_train)))
        X_test_3D = np.array(list(executor.map(embedding_fn_3D, X_test)))

    rows_all_nan_train = np.isnan(X_train_3D).all(axis=1)
    nan_indices_train = np.where(rows_all_nan_train)[0]

    rows_all_nan_test = np.isnan(X_test_3D).all(axis=1)
    nan_indices_test = np.where(rows_all_nan_test)[0]

    X_train_3D = np.delete(X_train_3D, nan_indices_train, axis=0)
    y_train_3D = np.delete(y_train, nan_indices_train, axis=0)
    X_test_3D = np.delete(X_test_3D, nan_indices_test, axis=0)
    y_test_3D = np.delete(y_test, nan_indices_test, axis=0)

    get_score(X_train_3D, y_train_3D, X_test_3D, y_test_3D, "r")

mu
****************
2D MODEL
****************
Regression Metrics:
MAE:  0.2097
RMSE: 0.4920
R²:   0.1800
****************
2D+3D MODEL
*****************
Regression Metrics:
MAE:  0.1649
RMSE: 0.3981
R²:   0.4632
zpve
****************
2D MODEL
****************
Regression Metrics:
MAE:  3455.5742
RMSE: 5771.5050
R²:   0.5808
****************
2D+3D MODEL
*****************
Regression Metrics:
MAE:  2656.3301
RMSE: 3990.1253
R²:   0.7997
energy_U0
****************
2D MODEL
****************
Regression Metrics:
MAE:  406.5026
RMSE: 1548.7342
R²:   0.1344
****************
2D+3D MODEL
*****************
Regression Metrics:
MAE:  163.4554
RMSE: 855.0506
R²:   0.7362
